In [2]:
library('dplyr')
library('ggplot2')

In [5]:
df = read.csv('data/titanic.csv', sep=',', dec='.')

## 1.1 Изучаем таблицу

Можно посмотреть на первые пять строк!

In [6]:
head(df,5)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,,S


Можно посмотреть на последние пять строк!

In [7]:
tail(df,5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
887,887,0,2,"Montvila, Rev. Juozas",male,27,0,0,211536,13.00,,S
888,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.00,B42,S
889,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NA,1,2,W./C. 6607,23.45,,S
890,890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30.00,C148,C
891,891,0,3,"Dooley, Mr. Patrick",male,32,0,0,370376,7.75,,Q


Можно посмотреть на число колонок, строк и размерность таблицы.

In [9]:
ncol(df)  # число колонок
nrow(df)  # число строк
dim(df)   # размерность таблицы

[1] 12

[1] 891

[1] 891  12

Можно бросить взор на нашу таблицу!

In [10]:
glimpse(df)

Observations: 891
Variables: 12
$ PassengerId <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, ...
$ Survived    <int> 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0...
$ Pclass      <int> 3, 1, 3, 1, 3, 3, 1, 3, 3, 2, 3, 1, 3, 3, 3, 2, 3, 2, 3...
$ Name        <fctr> Braund, Mr. Owen Harris, Cumings, Mrs. John Bradley (F...
$ Sex         <fctr> male, female, female, female, male, male, male, male, ...
$ Age         <dbl> 22, 38, 26, 35, 35, NA, 54, 2, 27, 14, 4, 58, 20, 39, 1...
$ SibSp       <int> 1, 1, 0, 1, 0, 0, 0, 3, 0, 1, 1, 0, 0, 1, 0, 0, 4, 0, 1...
$ Parch       <int> 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 5, 0, 0, 1, 0, 0...
$ Ticket      <fctr> A/5 21171, PC 17599, STON/O2. 3101282, 113803, 373450,...
$ Fare        <dbl> 7.2500, 71.2833, 7.9250, 53.1000, 8.0500, 8.4583, 51.86...
$ Cabin       <fctr> , C85, , C123, , , E46, , , , G6, C103, , , , , , , , ...
$ Embarked    <fctr> S, C, S, S, S, Q, S, S, S, C, S, S, S, S, S, S, Q, S, ...


Можно посмотреть по таблице основные характеристики.

In [16]:
summary(df[c('Sex', 'Age', 'Pclass', 'SibSp', 'Ticket') ])

     Sex           Age            Pclass          SibSp            Ticket   
 female:314   Min.   : 0.42   Min.   :1.000   Min.   :0.000   1601    :  7  
 male  :577   1st Qu.:20.12   1st Qu.:2.000   1st Qu.:0.000   347082  :  7  
              Median :28.00   Median :3.000   Median :0.000   CA. 2343:  7  
              Mean   :29.70   Mean   :2.309   Mean   :0.523   3101295 :  6  
              3rd Qu.:38.00   3rd Qu.:3.000   3rd Qu.:1.000   347088  :  6  
              Max.   :80.00   Max.   :3.000   Max.   :8.000   CA 2144 :  6  
              NA's   :177                                     (Other) :852  

## 1.2 Основные операции над таблицами и dplyr